## Install Dependencies

In [1]:
# attendance kernel
# Python 3.6
# Tensorflow
# opencv
# matplotlib

In [2]:
# !pip install tensorflow==2.4.1 opencv-python matplotlib

## Import Dependencies

In [3]:
# Siamese neural network - explain during presentation
# https://www.cs.cmu.edu/~rsalakhu/papers/oneshot1.pdf

In [4]:
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

In [5]:
# import tensorflow dependencies - functional API
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten

In [6]:
# setup paths
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

In [ ]:
# Make the directories
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)

### Collect positives and anchors

In [ ]:
# http://vis-www.cs.umass.edu/lfw/ to download dataset
# uncompress tar GZ Labelled Faces in the wild dataset
!tar -xf lfw.tar # this extracts it into the same place it's in

In [ ]:
# move LFW images to data/negative
for directory in os.listdir('lfw'):
    for file in os.listdir(os.path.join('lfw', directory)):
        EX_PATH = os.path.join('lfw', directory, file)
        NEW_PATH = os.path.join(NEG_PATH, file)
        os.replace(EX_PATH, NEW_PATH)

### Collect positive and anchor classes - neg images are 250px by 250px

In [7]:
# import uuid library to generate unique image names
import uuid

In [ ]:
# access webcam
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # cut frame to 250+250
    frame = frame[200:200+250,700:700+250, :]
    
    # collect anchors
    if cv2.waitKey(1) & 0XFF == ord('a'):
        imgname = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1())) # unique file path
        cv2.imwrite(imgname, frame) # write image
    
    # collect positives
    if cv2.waitKey(1) & 0XFF == ord('p'):
        imgname = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(imgname, frame) # write image
    
    cv2.imshow('Image Collection', frame) # show img to screen
    
    if cv2.waitKey(1) & 0XFF == ord('q'): # break gracefully, q closes down frame
        break
cap.release() # release webcam
cv2.destroyAllWindows() # close image show frame

In [ ]:
plt.imshow(frame)

In [ ]:
frame.shape # need 250x250

In [ ]:
plt.imshow(frame[:250,:250,:]) # top left corner...

In [ ]:
plt.imshow(frame[200:200+250,700:700+250, :]); # found it

In [ ]:
# after fix
plt.imshow(frame)

### get image directories

In [10]:
anchor = tf.data.Dataset.list_files(ANC_PATH + '/*.jpg').take(200)
positive = tf.data.Dataset.list_files(POS_PATH + '/*.jpg').take(200)
negative = tf.data.Dataset.list_files(NEG_PATH + '/*.jpg').take(200)

In [11]:
# Preprocessing - scale and resize
def preprocess(file_path):
    byte_img = tf.io.read_file(file_path) # read img
    img = tf.image.decode_jpeg(byte_img) # decode img
    img = tf.image.resize(img, [100, 100]) # resize img
    img=img/255.0 # scale img
    return img # return img

### create labelled dataset

In [33]:
# anchor positive = 1,1,1,1,1
# anchor negative = 0,0,0,0,0
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)

TypeError: object of type 'DatasetV1Adapter' has no len()

### build and train test partition

In [27]:
def preprocess_twin(input_img, validation_img, label):
    return (preprocess(input_img), preprocess(validation_img), label)